<a href="https://colab.research.google.com/github/146790g/Mode-Of-Action-Data/blob/master/MOA_MLP_2_Model_Fitting_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mode of Action 

##Mechanisms of Action (MoA) Prediction
##Can you improve the algorithm that classifies drugs based on their biological activity?

#（その２： Model Fitting - No. 3 ）

https://machinelearningmastery.com/multi-label-classification-with-deep-learning/



In [ ]:
%tensorflow_version 2.x

In [ ]:
%pwd
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!ls -lh '/content/gdrive/My Drive/Colab Notebooks/MOA/data'

total 649M
-rw------- 1 root root 3.2M Sep  2 06:04 sample_submission.csv
-rw------- 1 root root  25M Sep  2 06:04 test_features.csv
-rw------- 1 root root 150M Sep  2 06:04 train_features.csv
-rw------- 1 root root  19M Sep  2 06:04 train_targets_nonscored.csv
-rw------- 1 root root 9.7M Sep  2 06:04 train_targets_scored.csv
-rw------- 1 root root  48M Nov  4 11:42 X_fp_test
-rw------- 1 root root 285M Nov  4 11:42 X_fp_train
-rw------- 1 root root  38M Nov  4 11:42 y
-rw------- 1 root root  74M Nov  4 11:42 y0


# Module Load

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Adding iterative-stratification 
# Select add data from the right menu and search for iterative-stratification, then add it to your kernel.
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
#from iterstrat.ml_stratifiers import MultilabelStratifiedKFold


from time import time
import datetime
import gc
import pickle

import numpy as np
import pandas as pd 

# ML tools 
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
import tensorflow as tf 
from tensorflow.keras import layers
import tensorflow.keras.backend as K
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold
from tensorflow_addons.layers import WeightNormalization
# Setting random seeds
np.random.seed(42)
tf.random.set_seed(42)

# Visualization tools
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('white')
sns.set(font_scale=1.2)




# Data Load

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/content/gdrive/My Drive/Colab Notebooks/MOA/data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



data_dir = '/content/gdrive/My Drive/Colab Notebooks/MOA/data'

/content/gdrive/My Drive/Colab Notebooks/MOA/data/test_features.csv
/content/gdrive/My Drive/Colab Notebooks/MOA/data/sample_submission.csv
/content/gdrive/My Drive/Colab Notebooks/MOA/data/train_features.csv
/content/gdrive/My Drive/Colab Notebooks/MOA/data/train_targets_scored.csv
/content/gdrive/My Drive/Colab Notebooks/MOA/data/train_targets_nonscored.csv
/content/gdrive/My Drive/Colab Notebooks/MOA/data/X_fp_train
/content/gdrive/My Drive/Colab Notebooks/MOA/data/X_fp_test
/content/gdrive/My Drive/Colab Notebooks/MOA/data/y0
/content/gdrive/My Drive/Colab Notebooks/MOA/data/y


In [ ]:
with open(os.path.join(data_dir,'X_fp_train'),'rb') as web:
  X_fp_train=pickle.load(web)

   
with open(os.path.join(data_dir,'X_fp_test'),'rb') as web:
  X_fp_test=pickle.load(web)
    
with open(os.path.join(data_dir,'y'),'rb') as web:
  y=pickle.load(web)
    
with open(os.path.join(data_dir,'y0'),'rb') as web:
  y0=pickle.load(web)
    
X_fp_train=X_fp_train.drop('sig_id',axis=1)
X_fp_test=X_fp_test.drop('sig_id',axis=1)
y=y.drop('sig_id',axis=1)
y0=y0.drop('sig_id',axis=1)

display(X_fp_train)
print(X_fp_train.shape)
print(type(X_fp_train))

display(X_fp_test)
print(X_fp_test.shape)

display(y)
print(y.shape)

display(y0)
print(y0.shape)


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,...,feature_1523,feature_1524,feature_1525,feature_1526,feature_1527,feature_1528,feature_1529,feature_1530,feature_1531,feature_1532,feature_1533,feature_1534,feature_1535,feature_1536,feature_1537,feature_1538,feature_1539,feature_1540,feature_1541,feature_1542,feature_1543,feature_1544,feature_1545,feature_1546,feature_1547,feature_1548,feature_1549,feature_1550,feature_1551,feature_1552,feature_1553,feature_1554,feature_1555,feature_1556,feature_1557,feature_1558,feature_1559,cp_type,cp_time,cp_dose
0,1.134849,0.907687,-0.416385,-0.966814,-0.254723,-1.017473,-1.364787,-0.024007,0.679466,-0.311014,1.526019,0.171308,0.621302,-0.560444,0.281356,-1.058992,-1.139669,0.870454,0.379613,-0.511557,-0.723914,-1.330500,-1.676873,1.244529,0.560983,0.401883,0.240395,0.165332,-0.520720,0.773151,0.415307,-1.103744,-0.052653,-0.442339,-0.199741,0.273999,0.384531,-0.584221,-0.740294,-0.370812,...,0.870970,-0.732520,0.188052,0.742230,0.490607,-0.619474,0.068592,-1.504140,0.361238,-0.919899,0.224557,0.650906,-0.090966,-0.201176,-0.057078,-0.104565,0.012987,-0.319749,-0.779413,0.352420,0.882921,0.235929,0.534012,-0.141942,0.288712,-0.310691,0.742231,-0.429101,0.550838,-0.892721,0.135728,1.329258,0.522436,0.075909,0.300444,-0.958183,-0.222714,0,1,0
1,0.119282,0.681738,0.272399,0.080113,1.205169,0.686517,0.313396,0.555877,-0.538257,0.837592,-1.246895,-0.576216,-0.210328,0.545961,0.148988,0.393359,0.398284,0.397419,-0.838965,0.499417,1.226498,-0.136304,-0.396829,-0.444660,0.396746,-0.582440,0.633148,0.220342,-0.744141,-0.187325,-0.121009,-0.552709,1.658538,-0.370436,0.352198,-0.284901,0.202415,0.782016,-0.357596,-0.090577,...,0.783410,1.729174,0.987102,0.581350,-0.215986,-0.741784,0.195505,-0.566749,1.120088,-0.346712,0.500268,-1.208069,-0.773580,-0.739454,1.045011,-0.098501,0.002443,0.215311,-0.415526,0.047686,-0.154716,0.094627,-0.649273,-0.209533,0.653578,0.162298,0.077731,-0.683204,-0.247614,0.582455,-0.287878,-0.289009,0.235928,-1.066457,-0.229447,-0.269402,-0.404329,0,3,0
2,0.779973,0.946463,1.425350,-0.132928,-0.006122,1.492493,0.235577,0.365511,-0.006556,1.238615,-0.641087,-0.771401,-0.128997,-2.316078,0.879141,-0.546448,0.470107,-0.406391,-0.308666,-0.043430,-0.106817,-1.012321,-1.944864,0.532973,0.626072,0.698526,-1.378719,2.391957,-0.094196,1.580764,-1.458665,0.812531,0.737065,0.205563,-0.001697,-1.669590,0.147467,-0.155573,1.704209,1.307218,...,0.458180,1.309230,0.631910,-0.343990,-0.617886,-0.411689,0.289879,-0.749461,-0.244601,0.970306,-0.109021,-0.735008,-0.899690,0.112052,-1.295420,-0.153604,0.020435,0.372885,-0.649638,0.869034,-0.603512,-0.492646,0.027808,-0.587101,0.258990,-0.343727,0.510151,-1.153753,-0.087442,-1.067847,-0.548184,0.027069,-0.598075,0.229318,-0.456100,0.808609,-0.551953,0,2,0
3,-0.734910,-0.274641,-0.438509,0.759097,2.346330,-0.858153,-2.288417,0.309275,-0.192409,-1.383015,-1.000646,-0.495448,-1.143743,-0.817040,-1.763535,1.392512,-0.229131,-1.052197,0.244593,-2.122611,2.082587,-1.352449,-1.101286,-1.063392,-1.422739,0.002243,-1.052410,0.263509,-0.456821,-1.246761,-0.975328,0.124462,-2.086299,1.037352,0.779250,-2.137027,-1.410252,-1.056203,1.413863,-0.641700,...,0.290228,0.440013,-0.582151,-0.124091,0.085333,-0.035157,0.671462,-0.025099,-0.329930,-0.109062,0.359373,-0.453930,0.275757,0.016752,-0.304253,0.774766,0.351371,0.045686,-0.387079,0.273162,-0.143859,-0.159551,-0.241372,0.354303,-0.274247,0.033112,-0.285419,0.073812,-0.440913,-0.070533,0.216307,-0.034357,-0.516487,0.079811,0.000634,0.017973,-0.088352,0,2,0
4,-0.452718,-0.477513,0.972316,0.970731,1.463427,-0.869555,-0.375501,-0.2058

(23814, 1563)
<class 'pandas.core.frame.DataFrame'>


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,...,feature_1523,feature_1524,feature_1525,feature_1526,feature_1527,feature_1528,feature_1529,feature_1530,feature_1531,feature_1532,feature_1533,feature_1534,feature_1535,feature_1536,feature_1537,feature_1538,feature_1539,feature_1540,feature_1541,feature_1542,feature_1543,feature_1544,feature_1545,feature_1546,feature_1547,feature_1548,feature_1549,feature_1550,feature_1551,feature_1552,feature_1553,feature_1554,feature_1555,feature_1556,feature_1557,feature_1558,feature_1559,cp_type,cp_time,cp_dose
0,-0.755294,0.214796,-0.774511,0.705349,1.564580,-0.194968,-0.276944,0.363393,-0.444533,1.805324,-0.371021,-0.479669,1.294179,1.375143,0.051658,1.172079,-0.440924,-0.062191,-0.515950,-0.897857,-0.789403,0.331908,0.013708,0.731217,0.280174,0.726955,1.007077,0.758206,-0.463151,-0.205948,-0.729595,-0.068265,-0.926625,-0.757403,-1.410264,-0.018377,0.603804,0.831659,-0.407849,-1.489034,...,-0.155122,-0.128920,0.157413,-1.306263,0.272074,0.918754,-0.261455,0.015274,0.694862,-0.668645,-0.870018,1.456681,0.057016,-0.557674,-0.083250,0.286217,-0.392247,-0.528000,0.129146,-0.413125,-0.578525,-0.450043,-0.698387,-0.859095,0.219173,0.088578,0.542727,0.886659,0.060739,0.756908,1.240647,0.011547,-0.183502,1.374422,0.314476,-0.640933,-0.819822,0,1,0
1,-0.186773,0.374643,1.176402,-0.652299,-0.546638,-0.403447,-2.325903,0.645634,-0.485088,-0.048936,1.210023,0.419276,-1.275817,-0.425276,0.132044,-0.890770,-1.146702,-1.251700,-0.274084,-1.058294,-0.821554,1.029957,0.843801,0.088734,0.644651,-0.194701,-0.240824,1.799069,-1.067548,-0.836341,-0.634064,-0.140889,-0.423926,0.252623,-0.830175,1.399657,-1.112732,-1.097712,0.456949,0.012281,...,0.732828,0.470362,-0.323135,0.280814,0.332226,0.714134,-0.312355,1.156161,0.560084,0.352342,0.542858,-1.074762,-0.085142,0.416843,0.677727,0.230031,-0.037873,-0.192133,0.186839,0.797680,-0.091833,-0.156995,0.345948,-0.025300,-0.188768,0.113163,-0.047721,0.408430,-0.817178,-0.219832,0.355091,-1.072866,-0.336214,0.421539,0.762212,-0.324633,1.232997,0,3,0
2,0.351229,-0.155308,-0.613250,0.255053,-1.762152,0.333321,-0.434423,-0.484560,0.857189,0.514641,-0.132592,-0.834668,1.793695,-1.108673,1.197317,-0.086980,0.054445,1.397045,-1.129795,-1.353153,1.041231,-0.465987,-0.838953,0.284006,-0.377608,0.284396,0.769001,-0.964506,0.009738,0.077140,0.418757,-0.766888,-0.551620,-0.150097,-1.309084,0.404371,0.432542,0.741155,-1.588311,-1.132478,...,0.149081,0.178870,-0.538151,0.121730,-1.311772,0.024471,-1.422734,0.633899,-0.327280,0.487183,-2.135921,1.192886,0.203078,-0.095635,-0.426018,0.603836,-0.536776,-1.128896,-0.991975,0.257161,0.298067,-0.330981,0.540041,-1.008229,0.319691,-0.280347,1.218431,0.272635,1.088995,1.342968,1.908075,0.221462,-0.811315,0.256755,-1.140081,-1.442317,-0.298742,1,1,0
3,0.705453,0.318216,0.375618,0.678600,-0.881606,-1.173865,0.808203,-0.300028,1.126106,0.164837,1.758342,-1.455074,1.190058,-1.140901,0.148425,-0.513810,-1.344397,2.252051,1.739004,-0.134163,-0.752355,-0.965540,-0.979172,0.088734,-1.014836,1.144740,-0.175929,0.600419,1.338627,-0.696952,0.400296,0.683277,0.109186,0.321584,-0.834447,0.850169,0.545679,-0.068639,0.648665,-0.238637,...,0.454491,-0.350021,0.091848,-0.772585,0.367716,0.963815,1.064175,-0.428296,-0.375807,0.686538,0.624954,-0.148453,0.178884,-0.216934,-0.331103,0.611400,0.233651,0.302420,0.646369,-0.125181,-0.955064,1.909540,1.010591,-0.139492,0.676348,-1.183931,0.212428,-1.015779,0.009139,-0.603904,0.300780,0.370687,0.067473,-0.633106,-0.191964,0.997806,0.354367,0,1,1
4,-0.508836,-1.512891,1.649608,0.351728,-0.883199,-0.005990,0.701229,1.016848,0.332501,-1

(3982, 1563)


,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,...,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhibitor,retinoid_receptor_agonist,retinoid_receptor_antagonist,rho_associated_kinase_inhibitor,ribonucleoside_reductase_inhibitor,rna_polymerase_inhibitor,serotonin_receptor_agonist,serotonin_receptor_antagonist,serotonin_reuptake_inhibitor,sigma_receptor_agonist,sigma_receptor_antagonist,smoothened_receptor_antagonist,sodium_channel_inhibitor,sphingosine_receptor_agonist,src_inhibitor,steroid,syk_inhibitor,tachykinin_antagonist,tgf-beta_receptor_inhibitor,thrombin_inhibitor,thymidylate_synthase_inhibitor,tlr_agonist,tlr_antagonist,tnf_inhibitor,topoisomerase_inhibitor,transient_receptor_potential_channel_antagonist,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23810,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23811,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23812,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


(23814, 206)


,abc_transporter_expression_enhancer,abl_inhibitor,ace_inhibitor,acetylcholine_release_enhancer,adenosine_deaminase_inhibitor,adenosine_kinase_inhibitor,adenylyl_cyclase_inhibitor,age_inhibitor,alcohol_dehydrogenase_inhibitor,aldehyde_dehydrogenase_activator,aldose_reductase_inhibitor,alpha_mannosidase_inhibitor,ampk_inhibitor,androgen_biosynthesis_inhibitor,angiotensin_receptor_agonist,antacid,anthelmintic,antihypertensive,antimetabolite,antipruritic,antirheumatic_drug,antiseptic,antispasmodic,antithyroid_agent,antitussive,anxiolytic,ap_inhibitor,apolipoprotein_expression_enhancer,apoptosis_inhibitor,arf_inhibitor,aryl_hydrocarbon_receptor_agonist,aryl_hydrocarbon_receptor_antagonist,aspartic_protease_inhibitor,atherogenesis_inhibitor,atherosclerosis_formation_inhibitor,atp-sensitive_potassium_channel_agonist,atp-sensitive_potassium_channel_inhibitor,atp_channel_activator,atp_channel_blocker,atp_citrase_lyase_inhibitor,...,thromboxane_synthase_inhibitor,thyroid_hormone_inhibitor,thyroid_hormone_stimulant,thyrotropin_releasing_hormone_receptor_agonist,tie_inhibitor,tissue_transglutaminase_inhibitor,topical_anesthetic,topical_sunscreen_agent,trace_amine_associated_receptor_agonist,trace_amine_associated_receptor_antagonist,trail_modulator,transient_receptor_potential_channel_agonist,triacylglycerol_lipase_inhibitor,tricyclic_antidepressant,tryptophan_hydroxylase_inhibitor,tyrosinase_inhibitor,tyrosine_hydroxylase_inhibitor,tyrosine_phosphatase_inhibitor,ubiquitin-conjugating_enzyme_inhibitor,ubiquitin_ligase_inhibitor,urease_inhibitor,uric_acid_diuretic,uricase_inhibitor,uricosuric,urotensin_receptor_agonist,urotensin_receptor_antagonist,vasoconstrictor,vasodilator,vasopressin_receptor_agonist,vasopressin_receptor_antagonist,ve-cadherin_antagonist,vesicular_monoamine_transporter_inhibitor,vitamin_k_antagonist,voltage-gated_calcium_channel_ligand,voltage-gated_potassium_channel_activator,voltage-gated_sodium_channel_blocker,wdr5_mll_interaction_inhibitor,wnt_agonist,xanthine_oxidase_inhibitor,xiap_inhibitor
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23810,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23811,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23812,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


(23814, 402)


# Define Model

In [ ]:
X_fp_train.shape[1],y.shape[1]


(1563, 206)

In [ ]:
import tensorflow as tf
import numpy as np
from keras.models import Model
from keras.layers import Input, Lambda,Activation,Dropout,Flatten,Dense
import keras.backend as K
from tensorflow import keras

import tensorflow as tf
import numpy as np
from keras.models import Model
from keras.layers import Input, Lambda,Activation,Dropout,Flatten,Dense,BatchNormalization
import keras.backend as K
from tensorflow import keras,losses

p_min = 0.001
p_max = 0.999

from tensorflow.keras import regularizers

def logloss(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred,p_min,p_max)
    return -K.mean(y_true*K.log(y_pred) + (1-y_true)*K.log(1-y_pred))

def create_model(num_cols, hid_layers, activations, dropout_rate, lr, num_cols_y):
   
    inp1 = tf.keras.layers.Input(shape = (num_cols, ))
    x1 = tf.keras.layers.BatchNormalization()(inp1)

    for i, units in enumerate(hid_layers):
        x1 = tf.keras.layers.Dense(units, activation=activations[i])(x1)
        x1 = tf.keras.layers.Dropout(dropout_rate[i])(x1)
        x1 = tf.keras.layers.BatchNormalization()(x1)
    
    x1 = tf.keras.layers.Dense(num_cols_y,activation='sigmoid')(x1)
    model = tf.keras.models.Model(inputs= inp1, outputs= x1)
    
    model.compile(optimizer = 'adam',
                 loss='binary_crossentropy', metrics=['accuracy'])
    
    return model 

In [ ]:
feats = np.arange(0,1563,1)
inp_size = int(np.ceil(1* len(feats)))
print(inp_size)

1563


In [ ]:
 hid_layers = [[512, 768, 896],[384, 640, 1024],[768,768,896],[512, 384, 1024],
              [512, 640, 640], [640, 896, 1024], [256,640,896],[512,512,768],
              [512, 384, 896],[512,768,768]]

dropout_rate = [[0.65,0.35,0.35],[0.65,0.35,0.45],[0.7,0.4,0.4],[0.65,0.35,0.45],
                [0.65,0.35,0.45],[0.7,0.3,0.45],[0.7,0.35,0.4],[0.7,0.4,0.4],
               [0.7, 0.3, 0.4],[0.65, 0.3, 0.4]]

activations = [['elu', 'swish', 'selu'], ['selu','swish','selu'], ['selu','swish','selu'],
              ['selu','swish','elu'],['selu','swish','elu'],['elu','swish','selu'],
               ['selu','swish','elu'],['selu','elu','selu'],['selu','swish','selu'],
               ['selu','swish','elu']]

lr = [0.002989447175078026,0.003478591117397657,0.0033564715716489766,0.00377940659516057,
      0.00308613410674138,0.002893435735413818,0.003352025985912316,
      0.0037675761105464426, 0.0030559926758757876, 0.0040360434306992285]

feats = np.arange(0,1563,1)
inp_size = int(np.ceil(1* len(feats)))

model=create_model(1563,hid_layers[0], activations[0], dropout_rate[0], lr[0],206)

model.summary()


Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 1563)]            0         
_________________________________________________________________
batch_normalization_24 (Batc (None, 1563)              6252      
_________________________________________________________________
dense_24 (Dense)             (None, 512)               800768    
_________________________________________________________________
dropout_18 (Dropout)         (None, 512)               0         
_________________________________________________________________
batch_normalization_25 (Batc (None, 512)               2048      
_________________________________________________________________
dense_25 (Dense)             (None, 768)               393984    
_________________________________________________________________
dropout_19 (Dropout)         (None, 768)             

In [ ]:
results = list()
n_inputs, n_outputs = X.shape[1], y.shape[1]
	
	# define evaluation procedure
	#cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

X=X_fp_train2.astype('float64').values
y_np=y.astype('float64').values
  
kf=KFold(n_splits=4,shuffle=True,random_state=123)
 
tr_idx,var_idx = list(kf.split(X))[0]

tr_x,va_x = X[tr_idx],X[var_idx]
tr_y,va_y = y_np[tr_idx],y_np[var_idx]

display(tr_x)
print(tr_x.shape)

display(va_x)
print(va_x.shape)

display(tr_y)
print(tr_y.shape)

display(va_y)
print(va_y.shape)




array([[ 1.13484916,  0.90768744, -0.41638451, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.11928153,  0.68173822,  0.27239921, ...,  0.        ,
         3.        ,  0.        ],
       [ 0.77997254,  0.94646298,  1.42534985, ...,  0.        ,
         2.        ,  0.        ],
       ...,
       [ 0.52514873,  0.63122535,  0.28817292, ...,  1.        ,
         2.        ,  1.        ],
       [ 0.8164071 ,  0.4176183 ,  0.43163123, ...,  0.        ,
         1.        ,  0.        ],
       [-1.24309611,  1.56773029, -0.26957347, ...,  0.        ,
         3.        ,  0.        ]])

(17860, 1563)


array([[-0.07723442,  0.45386242, -0.79980709, ...,  0.        ,
         2.        ,  1.        ],
       [ 0.56154044,  2.15639416, -1.71022872, ...,  0.        ,
         3.        ,  0.        ],
       [-0.18992414, -0.20388731, -0.95615965, ...,  0.        ,
         2.        ,  0.        ],
       ...,
       [ 2.0751076 ,  0.74660291,  1.40922027, ...,  0.        ,
         3.        ,  1.        ],
       [-0.55651655, -1.28373779,  0.84598153, ...,  0.        ,
         2.        ,  0.        ],
       [-1.9110206 ,  0.58722771, -0.58841728, ...,  0.        ,
         1.        ,  1.        ]])

(5954, 1563)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

(17860, 206)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

(5954, 206)


In [ ]:
model=create_model(1563,hid_layers[0], activations[0], dropout_rate[0], lr[0],206)
model.summary()



Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 1563)]            0         
_________________________________________________________________
batch_normalization_28 (Batc (None, 1563)              6252      
_________________________________________________________________
dense_28 (Dense)             (None, 512)               800768    
_________________________________________________________________
dropout_21 (Dropout)         (None, 512)               0         
_________________________________________________________________
batch_normalization_29 (Batc (None, 512)               2048      
_________________________________________________________________
dense_29 (Dense)             (None, 768)               393984    
_________________________________________________________________
dropout_22 (Dropout)         (None, 768)             

In [ ]:
history=model.fit(
    tr_x,
    tr_y,
    epochs=100,
    batch_size=10000,
    verbose=1,
    validation_data=(
        va_x,va_y
    ))

Epoch 1/100
2/2 [==============================] - 0s 90ms/step - loss: 0.0131 - accuracy: 0.1918 - val_loss: 0.0155 - val_accuracy: 0.1213
Epoch 2/100
2/2 [==============================] - 0s 87ms/step - loss: 0.0130 - accuracy: 0.1947 - val_loss: 0.0155 - val_accuracy: 0.1204
Epoch 3/100
2/2 [==============================] - 0s 92ms/step - loss: 0.0131 - accuracy: 0.1918 - val_loss: 0.0155 - val_accuracy: 0.1196
Epoch 4/100
2/2 [==============================] - 0s 93ms/step - loss: 0.0130 - accuracy: 0.1944 - val_loss: 0.0155 - val_accuracy: 0.1189
Epoch 5/100
2/2 [==============================] - 0s 92ms/step - loss: 0.0130 - accuracy: 0.1946 - val_loss: 0.0155 - val_accuracy: 0.1194
Epoch 6/100
2/2 [==============================] - 0s 91ms/step - loss: 0.0129 - accuracy: 0.1963 - val_loss: 0.0155 - val_accuracy: 0.1191
Epoch 7/100
2/2 [==============================] - 0s 86ms/step - loss: 0.0129 - accuracy: 0.1978 - val_loss: 0.0155 - val_accuracy: 0.1192
Epoch 8/100
2/2 [===

In [ ]:
def evaluate_model(X, y):
	results = list()
	n_inputs, n_outputs = X.shape[1], y.shape[1]
	
	# define evaluation procedure
	#cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
  
	kf=KFold(n_splits=4,shuffle=True,random_state=123)

	# enumerate folds
	for train_ix, test_ix in cv.split(X):
		# prepare data
		X_train, X_test = X[train_ix], X[test_ix]
		y_train, y_test = y[train_ix], y[test_ix]
		# define model
		model = model=create_model(1563,hid_layers[0], activations[0], dropout_rate[0], lr[0],206)
		# fit model
		model.fit(X_train, y_train, verbose=1, epochs=300)
		# make a prediction on the test set
		yhat = model.predict(X_test)
		# round probabilities to class labels
		yhat = yhat.round()
		# calculate accuracy
		acc = accuracy_score(y_test, yhat)
		# store result
		print('>%.3f' % acc)
		results.append(acc)
	return results

In [ ]:
X_fp_train2=X_fp_train.dropna(axis=1)

X=X_fp_train2.astype('float64').values
y_np=y.astype('float64').values
print(X)
print(X_fp_train2)

print(X.shape)

[[ 1.13484916  0.90768744 -0.41638451 ...  0.          1.
   0.        ]
 [ 0.11928153  0.68173822  0.27239921 ...  0.          3.
   0.        ]
 [ 0.77997254  0.94646298  1.42534985 ...  0.          2.
   0.        ]
 ...
 [ 0.52514873  0.63122535  0.28817292 ...  1.          2.
   1.        ]
 [ 0.8164071   0.4176183   0.43163123 ...  0.          1.
   0.        ]
 [-1.24309611  1.56773029 -0.26957347 ...  0.          3.
   0.        ]]
       feature_0  feature_1  feature_2  ...  cp_type  cp_time  cp_dose
0       1.134849   0.907687  -0.416385  ...        0        1        0
1       0.119282   0.681738   0.272399  ...        0        3        0
2       0.779973   0.946463   1.425350  ...        0        2        0
3      -0.734910  -0.274641  -0.438509  ...        0        2        0
4      -0.452718  -0.477513   0.972316  ...        0        3        1
...          ...        ...        ...  ...      ...      ...      ...
23809   0.209361  -0.022389  -0.235888  ...        0       

In [ ]:
X=X_fp_train.astype('float64').values

display(X)
print(X.shape)

array([[ 1.13484916,  0.90768744, -0.41638451, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.11928153,  0.68173822,  0.27239921, ...,  0.        ,
         3.        ,  0.        ],
       [ 0.77997254,  0.94646298,  1.42534985, ...,  0.        ,
         2.        ,  0.        ],
       ...,
       [ 0.52514873,  0.63122535,  0.28817292, ...,  1.        ,
         2.        ,  1.        ],
       [ 0.8164071 ,  0.4176183 ,  0.43163123, ...,  0.        ,
         1.        ,  0.        ],
       [-1.24309611,  1.56773029, -0.26957347, ...,  0.        ,
         3.        ,  0.        ]])

(23814, 1563)


In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score

# evaluate model
X=X_fp_train.astype('float64').values
y_np=y.astype('float64').values


results = evaluate_model(X, y_np)


# summarize performance
print('Accuracy: %.3f (%.3f)' % (mean(results), std(results)))

Epoch 1/300
670/670 [==============================] - 3s 4ms/step - loss: 0.0528 - logloss: 0.0491
Epoch 2/300
670/670 [==============================] - 3s 4ms/step - loss: 0.0226 - logloss: 0.0189
Epoch 3/300
670/670 [==============================] - 3s 5ms/step - loss: 0.0210 - logloss: 0.0176
Epoch 4/300
670/670 [==============================] - 3s 5ms/step - loss: 0.0202 - logloss: 0.0169
Epoch 5/300
670/670 [==============================] - 3s 5ms/step - loss: 0.0196 - logloss: 0.0164
Epoch 6/300
670/670 [==============================] - 3s 5ms/step - loss: 0.0192 - logloss: 0.0160
Epoch 7/300
670/670 [==============================] - 3s 4ms/step - loss: 0.0190 - logloss: 0.0158
Epoch 8/300
670/670 [==============================] - 3s 4ms/step - loss: 0.0187 - logloss: 0.0155
Epoch 9/300
670/670 [==============================] - 3s 4ms/step - loss: 0.0186 - logloss: 0.0154
Epoch 10/300
670/670 [==============================] - 3s 5ms/step - loss: 0.0184 - logloss: 0.0152

KeyboardInterrupt: ignored

# Q.E.D. 